In [6]:
# import pandas as pd

# # Đọc dữ liệu từ file csv
# df = pd.read_csv('data_test_2.csv')

# # Tạo các cột trống
# columns = ['Team_Win']
# empty_df = pd.DataFrame(columns=columns)


# # Thêm các cột trống vào DataFrame ban đầu
# df = pd.concat([df, empty_df], axis=1)

# # Ghi DataFrame mới vào file csv
# df.to_csv('data_test_2.csv', index=False)


# **CRAW DỮ LIỆU**

## SỬ DỤNG CÁC THƯ VIỆN CẦN THIẾT ĐỂ CRAW DỮ LIỆU

In [7]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json

## ĐỌC CÁC LINKS ĐÃ LẤY ĐƯỢC TỪ getlink.ipynb

In [8]:
# Đọc file csv
df = pd.read_csv('./data_craw/data_add.csv')

# Lấy tất cả các link trong cột 'link'
links = df['link']
df.head()

,link
0,/match/oce/569654542
1,/match/oce/569652416
2,/match/oce/569647786
3,/match/oce/569650418
4,/match/oce/569649445


Khai báo một số trường cần thiết vào từ điển

In [9]:
lane_dict = {
    1: 'top_blue',
    2: 'top_red',
    3: 'jung_blue',
    4: 'jung_red',
    5: 'mid_blue',
    6: 'mid_red',
    7: 'ad_blue',
    8: 'ad_red',
    9: 'support_blue',
    10: 'support_red'
}

## ĐỊNH NGHĨA MỘT SỐ HÀM CẦN THIẾT ĐỂ CRAW DỮ LIỆU

### CÁC HÀM ĐỂ CRAW DỮ LIỆU TỪNG TRẬN ĐẤU TRONG LINK

In [10]:
def get_champion_name(soup):
    i = 0
    rt = []
    # Lọc ra các tags có chứa dữ liệu
    div_tags = soup.find_all('div', {'class': 'relative'})
    
    # Duyệt qua từng tag và kiểm tra nội dung
    for div_tag in div_tags:
        a_tag = div_tag.find('a')
        if a_tag:
            i= i+1
            img_tag = a_tag.find('img')
            if img_tag:
                champion_name = img_tag['alt']
                lane = lane_dict[i]
                rt.append((lane, champion_name))
    return rt

In [11]:
def get_Gold(soup):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        if tag.string and 'graphDD1' in tag.string:
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    return data[0]
    return None

In [12]:
def get_Turret(soup):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        if tag.string and 'graphDD41"' in tag.string :
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    blue = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    red = my_list2[15][1]
                    return blue, red
    return None

In [13]:
def get_KDA_Lane(soup, graph_ka, graph_d):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        if tag.string and graph_ka in tag.string :
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    ka_red = my_list1[15][1]
                    
                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    ka_blue = my_list2[15][1]
        if tag.string and graph_d in tag.string :
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    d_red = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    d_blue = my_list2[15][1]
    
    if (d_blue == 0): 
        d_blue = 1
    
    if (d_red == 0): 
        d_red = 1
    kda_blue = ka_blue/d_blue
    kda_red = ka_red/d_red
    return kda_blue, kda_red

In [14]:
def get_KDA_All_Lane(i, soup):
    kda_top_blue, kda_top_red = get_KDA_Lane(soup, 'graphDD11"', 'graphDD14"')
    kda_jungle_blue, kda_jungle_red = get_KDA_Lane(soup, 'graphDD69"', 'graphDD72"')
    kda_mid_blue, kda_mid_red = get_KDA_Lane(soup, 'graphDD127"', 'graphDD130"')
    kda_ad_blue, kda_ad_red = get_KDA_Lane(soup, 'graphDD185"', 'graphDD188"')
    kda_sp_blue, kda_sp_red = get_KDA_Lane(soup, 'graphDD243"', 'graphDD246"')
    df.loc[i, 'kda_top_blue'] = round(kda_top_blue, 2)
    df.loc[i, 'kda_top_red'] = round(kda_top_red, 2)
    df.loc[i, 'kda_jungle_blue'] = round(kda_jungle_blue, 2)
    df.loc[i, 'kda_jungle_red'] = round(kda_jungle_red, 2)
    df.loc[i, 'kda_mid_blue'] = round(kda_mid_blue, 2)
    df.loc[i, 'kda_mid_red'] = round(kda_mid_red, 2)
    df.loc[i, 'kda_ad_blue'] = round(kda_ad_blue, 2)
    df.loc[i, 'kda_ad_red'] = round(kda_ad_red, 2)
    df.loc[i, 'kda_sp_blue'] = round(kda_sp_blue, 2)
    df.loc[i, 'kda_sp_red'] = round(kda_sp_red, 2)

In [15]:
def get_Dragon_Count(soup):
    # Lọc ra các script tag có chứa dữ liệu
    script_tags = soup.find_all('script', {'type': 'text/javascript'})
    # Duyệt qua từng tag script và kiểm tra nội dung
    for tag in script_tags:
        # in tag.string and 'graphDD380' not in tag.string and 'graphDD382' not in tag.string and 'graphDD419' not in tag.string
        if tag.string and 'graphDD38"' in tag.string:
                data = re.findall(r'data:\s*(\[\[.*?\]\])', tag.string)
                if data:
                    a = data[0]
                    b = data[1]
                    my_string1 = ''.join(map(str, data[0]))
                    my_list1 = json.loads(my_string1)
                    blue = my_list1[15][1]

                    my_string2 = ''.join(map(str, data[1]))
                    my_list2 = json.loads(my_string2)
                    red = my_list2[15][1]
                    return blue, red
    return None

In [16]:
def get_Rift_Heralds_killed(soup):
    div_tag = soup.find('div', {'class': 'box matchBox'})
    th_blue_tags = div_tag.find_all('th', {'class': 'text-center no-padding-lateral'})
    
    blue_rift_heralds_tag = th_blue_tags[0].find('div', {'teamcounters-heralds-13'})
    blue_rift_heralds_value = blue_rift_heralds_tag.find_parent('div').text.strip()
    
    red_rift_heralds_tag = th_blue_tags[1].find('div', {'teamcounters-heralds-13'})
    red_rift_heralds_value = red_rift_heralds_tag.find_parent('div').text.strip()

    return blue_rift_heralds_value, red_rift_heralds_value

In [17]:
def get_teamWin(soup):
    teamWin = ""
    div_tag = soup.find('div', {'class': 'box matchBox'})
    th_blue_tag = div_tag.find('th', {'class': 'text-left no-padding-right'})
    
    blue_victory_tag = th_blue_tag.find('span', {'class': 'victory'})
    blue_defeat_tag = th_blue_tag.find('span', {'class': 'defeat'})

    if (blue_victory_tag is not None) and (blue_defeat_tag is None):
        teamWin = "blue"
    else: 
        if  (blue_victory_tag is None) and (blue_defeat_tag is not None):
            teamWin = "red"

    return teamWin

### THỰC HIỆN VIỆC CRAW DỮ LIỆU TRẬN ĐẤU

In [ ]:
# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome()

i = 0
# Duyệt qua từng link và lấy thông tin
for link in links:
    print(link)
    # Mở trang web
    driver.get('https://www.leagueofgraphs.com/'+ link)

    # Chờ cho trang web tải xong
    driver.implicitly_wait(15)

    # Lấy HTML của trang web
    html = driver.page_source

    # Phân tích HTML bằng BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    #Lấy tên tướng của 2 đội chơi
    lst = get_champion_name(soup)
    for my_tuple in lst:
        lane = my_tuple[0] # lấy phần tử đầu tiên trong tuple
        champion_name = my_tuple[1] # lấy phần tử thứ hai trong tuple
        df.loc[i,lane] = champion_name
    #Lấy dữ liệu chênh lệch vàng của 2 đội trước phút 15
    my_string = ''.join(map(str, get_Gold(soup)))
    my_list = json.loads(my_string)
    try:
        gold_difference = my_list[15][1]
    except IndexError:
        continue
    df.loc[i, 'gold_difference'] = gold_difference
    
    #Lấy KDA (kill/dead/assist) của từng người chơi trước phút 15
    get_KDA_All_Lane(i, soup)
    
    #Lấy dữ liệu chênh lệch phá trụ của 2 đội trước phút 15
    turret_blue, turret_red = get_Turret(soup)
    df.loc[i, 'turret_blue'] = turret_blue
    df.loc[i, 'turret_red'] = turret_red

    #Lấy dữ liệu chênh lệch bùa lợi rồng của 2 đội trước phút 15
    dragon_blue, dragon_red= get_Dragon_Count(soup)
    df.loc[i, 'dragon_blue'] = dragon_blue
    df.loc[i, 'dragon_red'] = dragon_red

    #Lấy dữ liệu chênh lệch bùa lợi sứ giả khe nứt của 2 đội trước phút 15
    blue_rift_heralds, red_rift_heralds = get_Rift_Heralds_killed(soup)
    df.loc[i, 'blue_heralds'] = blue_rift_heralds
    df.loc[i, 'red_heralds'] = red_rift_heralds
    
    #Lấy dữ liệu đội thắng trận đấu
    df.loc[i,'team_win'] = get_teamWin(soup)
    
    print(str(i)+ " done");
    i = i + 1
    #Ghi dữ liệu vào file
    df.to_csv('./data_craw/data_craw.csv', index=False)

# Đóng trình duyệt
driver.quit()


#### Dữ liệu trận đấu sau khi craw

In [34]:
df.head()

,link,top_blue,top_red,jung_blue,jung_red,mid_blue,mid_red,ad_blue,ad_red,support_blue,...,kda_sp_blue,kda_sp_red,gold_difference,turret_blue,turret_red,dragon_blue,dragon_red,blue_heralds,red_heralds,team_win
0,/match/oce/569654542,Camille,Singed,Warwick,Lee Sin,Lissandra,Ahri,Ezreal,Lucian,Sona,...,2.67,4.0,1289.0,1.0,0.0,1.0,0.0,0,2,blue
1,/match/oce/569652416,Miss Fortune,Qiyana,Annie,Aatrox,Kennen,Sivir,Graves,Yuumi,Rell,...,4.00,1.0,6398.0,0.0,0.0,0.0,2.0,2,0,blue
2,/match/oce/569647786,Malphite,Qiyana,Aphelios,Renekton,Viego,Lucian,Syndra,Milio,Lulu,...,0.20,12.0,-14315.0,1.0,0.0,2.0,0.0,0,1,red
3,/match/oce/569650418,Bel'Veth,Yone,Nidalee,Shaco,Graves,Gragas,Varus,Soraka,Blitzcrank,...,6.00,1.0,5717.0,0.0,1.0,0.0,1.0,1,1,blue
4,/match/oce/569649445,Kayle,Fiora,Viego,Fiddlesticks,Rumble,Zed,Jinx,Sivir,Lulu,...,5.00,0.0,3074.0,0.0,1.0,0.0,2.0,2,0,blue


### CÁC HÀM ĐỂ CRAW DỮ LIỆU VỀ TƯỚNG TRONG TRÒ CHƠI LIÊN MINH HUYỀN THOẠI

In [36]:
def get_champions(data):
    driver.get('https://www.leagueoflegends.com/en-sg/champions/')
    driver.implicitly_wait(15)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # Lấy thông tin các tag có chứa dữ liệu thông tin tên tướng
    all_a_tag = soup.find_all('a', {'class': 'style__Wrapper-n3ovyt-0 style__ResponsiveWrapper-n3ovyt-4 ehjaZK elVPdk style__Item-sc-13btjky-3 CanXV isVisible isFirstTime'})
    i =0
    for a_tag in all_a_tag:
        champion_name_tag = a_tag.find('span', {'style__Text-n3ovyt-3 gMLOLF'})
        champion_name = champion_name_tag.text.strip()
        data.loc[i, 'Champions'] = champion_name
        i= i + 1
        data.to_csv('./data_craw/data_craw_champions.csv', index=False)
    

In [37]:
def get_champions_Lane(data):
    list_lanes = ["support", "adc", "mid", "jungle", "top"]
    for list_lane in list_lanes:
        driver.get('https://www.op.gg/champions?region=global&tier=diamond_plus&position=' + list_lane)
        driver.implicitly_wait(15)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Lấy thông tin các tag có chứa dữ liệu thông tin vị trí tướng
        all_td_tags = soup.find_all('td', {'class': 'css-cym2o0 e1oulx2j6'})
        
        for tag in all_td_tags:
            champion_name_tag = tag.find('img', {'bg-image'})
            champion_name = champion_name_tag.find_parent('a').text.strip()
            data.loc[data["Champions"] == champion_name, "Lane"] = list_lane
            data.to_csv('./data_craw/data_craw_champions.csv', index=False)
    

In [38]:
def get_champions_role_diff(data):
    champion_name = data['Champions']
    for name in champion_name:
        name_valid = name.replace("Nunu & Willump", "nunu").replace("Renata Glasc", "renata").replace(". ","-").replace(" ", "-").replace("'", "-")
        driver.get('https://www.leagueoflegends.com/en-sg/champions/' + name_valid)
        driver.implicitly_wait(15)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Lấy thông tin các tag có chứa dữ liệu thông tin vai trò tướng
        div_tag = soup.find_all('div', {'class': 'style__SpecsItemValue-sc-8gkpub-15 kPaHxk'})
        champion_role = div_tag[0].text.strip()
        champion_difficulty = div_tag[1].text.strip()

        data.loc[data["Champions"] == name, "Role"] = champion_role
        data.loc[data["Champions"] == name, "Difficulty"] = champion_difficulty
        data.to_csv('./data_craw/data_craw_champions.csv', index=False)
    

### THỰC HIỆN VIỆC CRAW DỮ LIỆU THÔNG TIN TƯỚNG

In [39]:
# Khởi tạo trình duyệt Chrome
driver = webdriver.Chrome()

data = pd.read_csv('./data_craw/data_craw_champions.csv')
# Duyệt qua từng link và lấy thông tin

# Mở trang web
driver.get('https://www.leagueoflegends.com/en-sg/champions/')

# Chờ cho trang web tải xong
driver.implicitly_wait(15)

# Lấy HTML của trang web
html = driver.page_source

# Phân tích HTML bằng BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
# Lấy tên tất cả các tướng trong trò chơi
get_champions(data)

# Lấy vị trí đi đường của các tướng đó trong trò chơi
get_champions_Lane(data)

# Lấy vai trò và độ khó của các tướng đó trong trò chơi
get_champions_role_diff(data)

# Đóng trình duyệt
driver.quit()

#### Dữ liệu tướng sau khi craw

In [40]:
data.head()

,Champions,Lane,Role,Difficulty
0,Aatrox,top,Fighter,Moderate
1,Ahri,mid,Mage,Moderate
2,Akali,top,Assassin,Moderate
3,Akshan,top,Marksman,Low
4,Alistar,support,Tank,Moderate
